In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

batch_size = 20

# data augmentation configuration for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/home/kanishka/Projects/Ikea product classification/downloads/Train',  
        target_size=(150, 150), 
        batch_size=batch_size,
    shuffle = True,
        class_mode='categorical')  

validation_generator = validation_datagen.flow_from_directory(
        '/home/kanishka/Projects/Ikea product classification/downloads/Validation',
        target_size=(150, 150),
        batch_size=batch_size,
    shuffle = True,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        steps_per_epoch=1373 // batch_size,
        epochs=25,
        validation_data=validation_generator,
        validation_steps=333 // batch_size)


Using TensorFlow backend.
W0715 17:17:42.140891 140686747932480 deprecation_wrapper.py:119] From /home/kanishka/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 17:17:42.160212 140686747932480 deprecation_wrapper.py:119] From /home/kanishka/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 17:17:42.164270 140686747932480 deprecation_wrapper.py:119] From /home/kanishka/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0715 17:17:42.200224 140686747932480 deprecation_wrapper.py:119] From /home/kanishka/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use 

Found 1373 images belonging to 4 classes.
Found 333 images belonging to 4 classes.


W0715 17:17:42.880936 140686747932480 deprecation.py:323] From /home/kanishka/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/25
22/68 [========>.....................] - ETA: 28s - loss: 1.6326 - acc: 0.2977

/home/kanishka/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


68/68 [==============================] - 39s 581ms/step - loss: 1.4181 - acc: 0.3772 - val_loss: 1.2433 - val_acc: 0.4844
Epoch 2/25
68/68 [==============================] - 37s 540ms/step - loss: 1.0540 - acc: 0.5657 - val_loss: 1.1005 - val_acc: 0.5751
Epoch 3/25
68/68 [==============================] - 37s 538ms/step - loss: 0.9733 - acc: 0.5966 - val_loss: 1.0796 - val_acc: 0.5719
Epoch 4/25
68/68 [==============================] - 37s 550ms/step - loss: 0.9544 - acc: 0.6139 - val_loss: 1.0699 - val_acc: 0.5655
Epoch 5/25
68/68 [==============================] - 38s 552ms/step - loss: 0.9567 - acc: 0.6303 - val_loss: 1.0301 - val_acc: 0.6070
Epoch 6/25
68/68 [==============================] - 38s 552ms/step - loss: 0.9013 - acc: 0.6518 - val_loss: 1.0458 - val_acc: 0.5655
Epoch 7/25
68/68 [==============================] - 36s 533ms/step - loss: 0.8995 - acc: 0.6454 - val_loss: 1.0370 - val_acc: 0.5942
Epoch 8/25
68/68 [==============================] - 38s 554ms/step - loss: 0.896

In [56]:
model.save('myModel.h5')
model.save_weights('Ikea_classification_weights.h5')

In [50]:
datagen = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2)
test_data = datagen.flow_from_directory('/home/kanishka/Projects/Ikea product classification/downloads/Test',
                                       target_size = (150, 150),
                                       batch_size = batch_size, shuffle = False)
acc = model.evaluate_generator(test_data, 48//batch_size)[1]
print("Accuracy: %.2f"%(acc*100)+"%")

Found 48 images belonging to 4 classes.
Accuracy: 82.50%


In [55]:
from sklearn.metrics import confusion_matrix, classification_report
Test_pred = model.predict_generator(test_data, 48//batch_size+1)
test_pred = np.argmax(Test_pred, axis =1)

print ('CONFUSION MATRIX\n',confusion_matrix(test_data.labels, test_pred))

CONFUSION MATRIX
 [[11  0  0  1]
 [ 0 11  0  1]
 [ 1  0  6  5]
 [ 0  0  1 11]]
